# Подготовка данных и модель для env'а

### Импорты

In [ ]:
from typing import Optional
import numpy as np
import gymnasium as gym
import math
import torch.optim as optim
import torch.nn as nn
import torch
import torchvision
import torchvision.transforms as transforms
# import torch.nn.functional as F
from sb3_contrib import MaskablePPO
# from sb3_contrib.common.envs import InvalidActionEnvDiscrete
from sb3_contrib.common.maskable.evaluation import evaluate_policy
from sb3_contrib.common.maskable.utils import get_action_masks
# from sb3_contrib.common.maskable.policies import MaskableActorCriticPolicy
from sb3_contrib.common.wrappers import ActionMasker
# from sb3_contrib.common.maskable.callbacks import MaskableEvalCallback
from stable_baselines3 import DQN
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env

from hpo_rl.models.simple_cnn import SimpleCNN 
from hpo_rl.environments.OnlineConveyorDiscreteMaskedEnv import OnlineConveyorDiscreteMaskedEnv



### Подготовка датасета (MNIST)

In [17]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,), (0.5,))])

trainset = torchvision.datasets.MNIST(root='./data',
                                      train=True,
                                      download=True,
                                      transform=transform)

train_loader = torch.utils.data.DataLoader(trainset,
                                           batch_size=64,
                                           shuffle=True,
                                           num_workers=2)

dataiter = iter(train_loader)
images, labels = next(dataiter)

print('Shape of a batch of images:', images.shape)
print('Shape of a batch of labels:', labels.shape)

Shape of a batch of images: torch.Size([64, 1, 28, 28])
Shape of a batch of labels: torch.Size([64])


### Модель для теста env'а

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
model = SimpleCNN()
model = model.to(device)
print(model)

cpu
SimpleCNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=3136, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)


### Конфиг для env'а

In [ ]:
test_config = {
    "model_type": "cnn",
    "hyperparameters": {
        0: {
            "name": "optimizer",
            "options": [optim.Adam, optim.SGD]
        },
        1: {
            "name": "learning_rate",
            "options": [0.001, 0.01, 0.1]
        },
        2: {
            "name": "batch_size",
            "options": [32, 64, 128]
        },
        3: {
            "name": "number_epochs",
            "options": [1]
        },
        4: {
            "name": "criterion",
            "options": [nn.CrossEntropyLoss()]
        }
    }
}

# Env'ы

## Конвейер Дискретный с масками

### Сам env

TODO: 
- Code review. 
- Снести кусок кода с обучением модели в отдельную функцию. 
- Придумать, что делать с info, ибо он пустой (я знаю, что там сейчас возвращается маска, но это можно убрать и ничего не должно поменяться). 
- Поменять train_loader на более общий датасет, если будем ещё модели имплементировать.
- Обернуть в vector_env для параллельных вычисленый
- PEP8
- Поэксперементировать с observation_space
- Можно попробовать сделать из interpret_hyperparams словарь и распаковывать его в step

In [ ]:
class OnlineEnvConveyorDiscreteMasked(gym.Env):
    def __init__(self, config, model, train_loader):
        super().__init__()
        
        # Инициализация input'а
        self.train_loader = train_loader
        self.hyperparameter_space = config["hyperparameters"]
        self.model_type = config["model_type"]
        if self.model_type not in ["cnn"]:
            raise Exception("Unknown model type")
        self.model = model

        # Максимальная длина массива гиперпараметров в конфиге
        max_hyperparam_len = max([len(hyp["options"]) for hyp in self.hyperparameter_space.values()])

        # Все маски, где маска - массив из значений 0 и 1, где 1 отвечает за то, что действие можно сделать, а 0 - нет
        # В данном случае маска отвечает за количество валидных значений гиперпараметров
        # Например, если максимальное значение валидных значений 5, а для данного гиперпараметра - 2,
        # то получится массив [1,1,0,0,0]
        self.all_masks = [(np.arange(max_hyperparam_len) < len(hyp["options"])).astype(np.int8) for hyp in self.hyperparameter_space.values()]

        # Нынешний шаг
        self.cur_step = 0
        # Массив со значениями в виде индексов в hyperparameter_space для соответствующих гиперпараметров 
        # (hyp_setup[i] - значение гиперпараметра в hyperparameter_space, i - номер гиперпараметра)
        self.hyp_setup = np.zeros(len(self.hyperparameter_space), dtype=np.int32)
        # Нынешняя маска
        self.cur_mask = self.all_masks[0]
        
        # За действие берем выбор значения гиперпараметра из hyperparameter_space (контролируется маской)
        self.action_space = gym.spaces.Discrete(max_hyperparam_len)
        # Агенту показываем маски и текущий набор гиперпараметров
        self.observation_space = gym.spaces.Dict({
            "hyperparameters": gym.spaces.Box(low=0, high=max_hyperparam_len, shape=(len(self.hyperparameter_space),), dtype=np.int32),
            "action_mask":  gym.spaces.Box(low=0, high=1, shape=(max_hyperparam_len,), dtype=np.int32)
            })

    def _get_obs(self):
        return {"hyperparameters": self.hyp_setup, "action_mask": self.cur_mask}
    
    def _get_info(self):
        return {"action_mask": self.cur_mask}
    
    def step(self, action):    
        # Идем в reset, если полностью выбрали все гиперпараметры
        terminated = self.cur_step >= len(self.hyp_setup)
        # Костыль, чтобы не выходить за границы массива
        if not terminated:
            # Присваевыаем выбор гиперпараметра в массив геперпараметров
            self.hyp_setup[self.cur_step] = action
        if self.model_type == "cnn":
            # Если сделали набор гиперпараметров - интерпретируем наш выбор и обучаем модель 
            if terminated:
                optimizer, learning_rate, batch_size, number_epochs, criterion = self.interpret_hyperparams()
                optimizer = optimizer(self.model.parameters(), lr=learning_rate)
                # Логика обучения модели
                for epoch in range(number_epochs):  # В STEP НЕ ПЕРЕОПРЕДЕЛЯЕТСЯ МОДЕЛЬ - ОГРОМНАЯ ОШИБКА
                    running_loss = 0.0
                    for i, data in enumerate(self.train_loader, 0):
                        inputs, labels = data
                        device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') # каждый раз проверять не стоит, пожалуй
                        inputs, labels = inputs.to(device), labels.to(device)

                        optimizer.zero_grad()

                        outputs = self.model(inputs)
                        loss = criterion(outputs, labels)
                        loss.backward()
                        optimizer.step()

                        running_loss += loss.item()
                        if i % batch_size == batch_size-1:
                            running_loss = 0.0 # чаво???
                print(f"CNN training finished. Final loss: {running_loss}")
                # Награда: -loss
                reward = -running_loss # собирать ревард из лоссов на трейне конечно не очень хорошо
            else:
                # Если всё ещё составляем массив значений гиперпараметров, то ничего не делаем
                reward = 0
        # Инкрементируем шаг
        self.cur_step+=1
        # Костыль, чтобы не выходить за границы массива
        if self.cur_step < len(self.all_masks):
            # Инкрементируем маску
            self.cur_mask = self.all_masks[self.cur_step]
        # Возвращаем наблюдение для агента и информацию для нас
        observation = self._get_obs()
        info = self._get_info()
        # Нужно для стандарта
        truncated = False
        return observation, reward, terminated, truncated, info
    
    # !!! options: Optional[dict] = None нужен для стандарта
    def reset(self, seed: Optional[int] = None, options: Optional[dict] = None):
        super().reset(seed=seed)
        
        # Возвращаем массив со значениями гиперпараметров, массив масок и шаг к изначальным значениям
        self.hyp_setup = np.zeros(len(self.hyperparameter_space), dtype=np.int32)
        self.cur_mask = self.all_masks[0]
        self.cur_step = 0

        # Возвращаем наблюдение для агента и информацию для нас
        observation = self._get_obs()
        info = self._get_info()
        return observation, info
    
    # !!! Нужно, чтобы алгоритм видел маски
    def action_masks(self):
        return self.cur_mask

    def interpret_hyperparams(self):
        if self.model_type == "cnn":
            # i - индес гиперпараметра, hyp_i - индекс значения гиперпараметра
            for i, hyp_i in enumerate(self.hyp_setup):
                name = self.hyperparameter_space[i]["name"]
                hyp = self.hyperparameter_space[i]["options"][hyp_i]
                if name == "optimizer":
                    optimizer = hyp
                elif name == "learning_rate":
                    learning_rate = hyp
                elif name == "batch_size":
                    batch_size = hyp
                elif name == "number_epochs":
                    number_epochs = hyp
                elif name == "criterion":
                    criterion = hyp
            return optimizer, learning_rate, batch_size, number_epochs, criterion

### check_env для дебага (То, что там IndexError - нормально, так как check_env не видит масок)

In [ ]:
env = OnlineConveyorDiscreteMaskedEnv(config=test_config, model=model, train_loader=train_loader)

check_env(env)

IndexError: list index out of range

### Модель для обучения

Данный env работает только с алгоритмами, которые поддреживают маски (В stable_baselines(stable_contrib) есть только PPO, который выполняет данное условие)

In [ ]:
def mask_fn(env: gym.Env) -> np.ndarray:
    return env.action_masks()

env = OnlineEnvConveyorDiscreteMasked(config=test_config,model=model,train_loader=train_loader)
env = ActionMasker(env, mask_fn) 
PPOmodel = MaskablePPO("MultiInputPolicy", env, gamma=0.4, seed=32, verbose=1, n_steps = 6)
PPOmodel.learn(total_timesteps=5000, log_interval=1)

# evaluate_policy(PPOmodel, env, n_eval_episodes=20, reward_threshold=90, warn=False)

# obs, _ = env.reset()
# while True:
#     action_masks = get_action_masks(env)
#     action, _states = PPOmodel.predict(obs, action_masks=action_masks)
#     obs, reward, terminated, truncated, info = env.step(action)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
CNN training finished. Final loss: 23.01479983329773
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 6        |
|    ep_rew_mean     | -23      |
| time/              |          |
|    fps             | 0        |
|    iterations      | 1        |
|    time_elapsed    | 16       |
|    total_timesteps | 6        |
---------------------------------
CNN training finished. Final loss: 96.99524450302124
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 6             |
|    ep_rew_mean          | -60           |
| time/                   |               |
|    fps                  | 0             |
|    iterations           | 2             |
|    time_elapsed         | 33            |
|    total_timesteps      | 12            |
| train/                  |               |
|    approx_kl      

KeyboardInterrupt: 

## Конвейер непрерывный

### Сам env

TODO: 
- Code review. 
- Снести кусок кода с обучением модели в отдельную функцию. 
- Придумать, что делать с info, ибо он пустой (я знаю, что там сейчас возвращается маска, но это можно убрать и ничего не должно поменяться). 
- Поменять train_loader на более общий датасет, если будем ещё модели имплементировать.
- Обернуть в vector_env для параллельных вычисленый
- PEP8
- Поэксперементировать с observation_space
- Поэксперементировать с interpret_action для непрерывних действий

In [ ]:
class OnlineEnvConveyorContinous(gym.Env):
    def __init__(self, config, model, train_loader):
        super().__init__()
        
        # Инициализация input'а
        self.hyperparameter_space = config["hyperparameters"]
        self.model_type = config["model_type"]
        self.model = model
        self.train_loader = train_loader
        # Нынешний шаг
        self.cur_step = 0
        # Массив со значениями в виде индексов в hyperparameter_space для соответствующих гиперпараметров 
        # (hyp_setup[i] - значение гиперпараметра в hyperparameter_space, i - номер гиперпараметра)
        self.hyp_setup = np.zeros(len(self.hyperparameter_space))
        # Словарь, в который записываются действия в формате "Название гиперпараметра": "Значение гиперпарамета"
        self.action_dict = {}
        # Делам действие непрерывно от 0 до 1
        self.action_space = gym.spaces.Box(low=0, high=1, shape=(1,) , dtype=np.float32)
        # Агенту показываем шаг и текущий набор гиперпараметров
        self.observation_space = gym.spaces.Dict({
            "hyperparameters": gym.spaces.Box(low=-1, high=float("inf"), shape=(len(self.hyperparameter_space),), dtype=np.int32),
            "step": gym.spaces.Box(low=0, high=len(self.hyperparameter_space), shape=(1,), dtype=np.int32)
            })

    def _get_obs(self):
        return {"hyperparameters": self.hyp_setup, 
                "step": np.array([self.cur_step], dtype=np.int32)}
    
    def _get_info(self):
        return {}
    
    def step(self, action):
        # Идем в reset, если полностью выбрали все гиперпараметры
        terminated = self.cur_step >= len(self.hyp_setup)
        # Костыль, чтобы не выходить за границы массива
        if not terminated:
            # Интерпретируем действие
            self.hyp_setup[self.cur_step] = self.interpret_action(action)
        if self.model_type == "cnn":
            if terminated:
                # Придериживаемся строго порядка ключей
                key_order = ["optimizer", "learning_rate", "batch_size", "number_epochs", "criterion"]
                optimizer, learning_rate, batch_size, number_epochs, criterion = (self.action_dict[key] for key in key_order)
                # Логика обучения модели
                optimizer = optimizer(self.model.parameters(), lr=learning_rate)
                for epoch in range(number_epochs):  
                    running_loss = 0.0
                    for i, data in enumerate(self.train_loader, 0):
                        inputs, labels = data

                        optimizer.zero_grad()

                        outputs = self.model(inputs)
                        loss = criterion(outputs, labels)
                        loss.backward()
                        optimizer.step()

                        running_loss += loss.item()
                        if i % batch_size == batch_size-1:
                            running_loss = 0.0 
                print(f"CNN training finished. Final loss: {running_loss}")
                # Награда: -loss
                reward = -running_loss
            else:
                # Если всё ещё составляем массив значений гиперпараметров, то ничего не делаем
                reward = 0
        # Инкрементируем шаг
        self.cur_step+=1
        # Возвращаем наблюдение для агента и информацию для нас
        observation = self._get_obs()
        info = self._get_info()
        # Нужно для стандарта
        truncated = False
        return observation, reward, terminated, truncated, info
        
    def reset(self, seed: Optional[int] = None):
        super().reset(seed=seed)
        # Возвращаем массив со значениями гиперпараметров, шаг и словарь с действиями к изначальным значениям
        self.hyp_setup = np.zeros(len(self.hyperparameter_space), dtype=np.int32)
        self.cur_step = 0
        self.action_dict = {}
        # Возвращаем наблюдение для агента и информацию для нас
        observation = self._get_obs()
        info = self._get_info()
        return observation, info
    
    def interpret_action(self, action):
        if self.model_type == "cnn":
            name = self.hyperparameter_space[self.cur_step]["name"]
            # Если действие дискретное, то делаем данное преобразование [0,1] -> [0, n], 
            # где n - размер массива возможных значений гиперпараметра
            if name in ["optimizer", "batch_size", "number_epochs", "criterion"]:
                num_options = len(self.hyperparameter_space[self.cur_step]["options"])
                # Минимум нужен для обработки крайнего значения, ибо он может выйти за грань массива
                filtered_action = min(math.floor(num_options * action[0]), num_options - 1)
                # Записываем в словарь действий
                self.action_dict[name] = self.hyperparameter_space[self.cur_step]["options"][filtered_action]
                return filtered_action
            # Если действие неперывние, то возвращаем просто значение в отрезке [0, 1]
            elif name == "learning_rate":
                lr_value = float(action[0])
                self.action_dict[name] = lr_value
                return lr_value

### check_env для дебага

In [ ]:
env = OnlineEnvConveyorContinous(config=test_config, model=model, train_loader=train_loader)

check_env(env)

### Модель для обучения

Данный env работает только с алгоритмами, где есть поддрежка непрерывных действий

In [ ]:
env = OnlineEnvConveyorContinous(config=test_config, model=model, train_loader=train_loader)
PPOmodel = PPO("MultiInputPolicy", env, verbose=1)
PPOmodel.learn(total_timesteps=10000, log_interval=4)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
CNN training finished. Final loss: 96.66686701774597


KeyboardInterrupt: 

## "Лаборатория"

### Сам env

TODO: 
- Code review. 
- Снести кусок кода с обучением модели в отдельную функцию. 
- Придумать, что делать с info, ибо он пустой (я знаю, что там сейчас возвращается маска, но это можно убрать и ничего не должно поменяться). 
- Поменять train_loader на более общий датасет, если будем ещё модели имплементировать.
- Обернуть в vector_env для параллельных вычисленый
- PEP8
- Поэксперементировать с observation_space
- Можно попробовать сделать из interpret_hyperparams словарь и распаковывать его в step
- Разобраться с np.concatinate
- Поиграться с наградами

In [ ]:
class OnlineEnvLab(gym.Env):
    def __init__(self, config, model, train_loader):
        super().__init__()
        
        # Инициализация input'а
        self.train_loader = train_loader
        self.hyperparameter_space = config["hyperparameters"]
        self.model_type = config["model_type"]
        if self.model_type not in ["cnn"]:
            raise Exception("Unknown model type")
        self.model = model
        
        # Максимальная длина массива гиперпараметров в конфиге
        max_hyperparam_len = max([len(hyp["options"]) for hyp in self.hyperparameter_space.values()])

        # В данном env'е маска сложная (состоит из нескольких подмасок). 
        # Все маски, где маска - массив из значений 0 и 1, где 1 отвечает за то, что действие можно сделать, а 0 - нет
        # Данная маска отвечает за количество валидных значений гиперпараметров
        # Например, если максимальное значение валидных значений 5, а для данного гиперпараметра - 2,
        # то получится массив [1,1,0,0,0]
        self.all_masks_for_hyp_value = [(np.arange(max_hyperparam_len) < len(hyp["options"])).astype(np.int32) for hyp in self.hyperparameter_space.values()]
        
        # В данной переменной будет лежать номер гиперпараметра, для которго мы будем выбирать значение     
        # Это нужно для того, чтобы поддерживать несколько масок одновременно
        self.pending_hyp = None

        # Массив со значениями в виде индексов в hyperparameter_space для соответствующих гиперпараметров 
        # (hyp_setup[i] - значение гиперпараметра в hyperparameter_space, i - номер гиперпараметра)
        self.hyp_setup = np.zeros(len(self.hyperparameter_space), dtype=np.int32)
        
        self.cur_mask_for_hyp_value = self.all_masks_for_hyp_value[0]
        # Инициализируем маску для действий, которые агент может делать (эти значения будут зануляться по мере работы агента)
        self.cur_mask_for_choose_hyp = np.ones(len(self.hyperparameter_space), dtype=np.int32)
        # Маска для остановки (всегда 1)
        self.mask_for_stop = np.array([1], dtype=np.int32)

        total_mask_size = len(self.cur_mask_for_choose_hyp) + len(self.cur_mask_for_hyp_value) + 1
        
        # За действие берем выбор значения гиперпараметра из hyperparameter_space, выбор действия и остановку (всё контролируется масками)
        self.action_space = gym.spaces.MultiDiscrete([len(self.hyperparameter_space), max_hyperparam_len, 1])
        # Агенту показываем маски и текущий набор гиперпараметров
        self.observation_space = gym.spaces.Dict({
            "hyperparameters": gym.spaces.Box(low=0, high=max_hyperparam_len, shape=(len(self.hyperparameter_space),), dtype=np.int32),
            "action_mask":  gym.spaces.Box(low=0, high=1, shape=(total_mask_size,), dtype=np.int32)
            })

    def _get_obs(self):
        # !!! Тут опасный мемент с concatinate, ибо оно с ним работает, и все имплементации, которые я смотрел делают по сути flatten для MultiDiscrete action'а
        full_mask = np.concatenate([self.cur_mask_for_choose_hyp, self.cur_mask_for_hyp_value, self.mask_for_stop])

        return {"hyperparameters": self.hyp_setup, "action_mask": full_mask}
    
    def _get_info(self):
        return {}
    
    # !!! options: Optional[dict] = None нужен для стандарта
    def step(self, action):
        choose_hyp, hyp_value, stop = action

        # Логика действий менятеся каждые два шага. 
        # На первом шаге мы задаем какой гиперпараметр будем менять и, соответственно, задаем маску
        # На втором мы задаем значение гиперпараметра
        # Это нужно, чтобы маска соответствовала каждому нашему действию, а не отставала на 1 шаг

        # 1 шаг
        if self.pending_hyp is None:
            if stop == 1:
                terminated = True
                reward = 0.0
                obs = self._get_obs()
                info = self._get_info()
                return obs, reward, terminated, False, info

            self.pending_hyp = int(choose_hyp)
            self.cur_mask_for_hyp_value = self.all_masks_for_hyp_value[self.pending_hyp]
            reward = -0.01  
            obs = self._get_obs()
            info = self._get_info()
            return obs, reward, False, False, info

        # Второй шаг
        chosen = int(self.pending_hyp)

        if stop == 1:
            terminated = True
            reward = 0.0
            self.pending_hyp = None
            obs = self._get_obs()
            info = self._get_info()
            return obs, reward, terminated, False, info

        self.hyp_setup[chosen] = int(hyp_value)
        self.cur_mask_for_choose_hyp[chosen] = 0
        self.pending_hyp = None
        self.cur_mask_for_hyp_value = np.zeros_like(self.cur_mask_for_hyp_value)  

        terminated = np.all(self.cur_mask_for_choose_hyp == 0)

        if terminated or stop == 1:
            # Если сделали набор гиперпараметров - интерпретируем наш выбор и обучаем модель 
            optimizer, learning_rate, batch_size, number_epochs, criterion = self.interpret_hyperparams()
            optimizer = optimizer(self.model.parameters(), learning_rate)
            # Логика обучения модели
            for epoch in range(number_epochs):  
                running_loss = 0.0
                for i, data in enumerate(self.train_loader, 0):
                    inputs, labels = data

                    optimizer.zero_grad()
                    outputs = self.model(inputs)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()

                    running_loss += loss.item()
                    if i % batch_size == batch_size-1:
                        running_loss = 0.0 
            print(f"CNN training finished. Final loss: {running_loss}")
            reward = -running_loss
        else:
            reward = -0.1
        observation = self._get_obs()
        info = self._get_info()
        truncated = False
        return observation, reward, terminated, truncated, info
    
    def reset(self, seed: Optional[int] = None, options: Optional[dict] = None):
        super().reset(seed=seed)
        
        # Возвращаем массив со значениями гиперпараметров, массив масок и шаг к изначальным значениям
        self.pending_hyp = None
        self.hyp_setup = np.zeros(len(self.hyperparameter_space), dtype=np.int32)
        self.cur_mask_for_hyp_value = self.all_masks_for_hyp_value[0]
        self.cur_mask_for_choose_hyp = np.ones(len(self.hyperparameter_space), dtype=np.int32)

        # Возвращаем наблюдение для агента и информацию для нас
        observation = self._get_obs()
        info = self._get_info()
        return observation, info
    
    # !!! Нужно, чтобы алгоритм видел маски
    def action_masks(self):
        return np.concatenate([self.cur_mask_for_choose_hyp, self.cur_mask_for_hyp_value, self.mask_for_stop])

    def interpret_hyperparams(self):
        # i - индес гиперпараметра, hyp_i - индекс значения гиперпараметра
        if self.model_type == "cnn":
            for i, hyp_i in enumerate(self.hyp_setup):
                name = self.hyperparameter_space[i]["name"]
                hyp = self.hyperparameter_space[i]["options"][hyp_i]
                if name == "optimizer":
                    optimizer = hyp
                elif name == "learning_rate":
                    learning_rate = hyp
                elif name == "batch_size":
                    batch_size = hyp
                elif name == "number_epochs":
                    number_epochs = hyp
                elif name == "criterion":
                    criterion = hyp
            return optimizer, learning_rate, batch_size, number_epochs, criterion

### check_env для дебага (То, что там IndexError - нормально, так как check_env не видит масок)

In [27]:
env = OnlineEnvLab(config=test_config, model=model, train_loader=train_loader)

check_env(env)

### Модель для обучения

Данный env работает только с алгоритмами, которые поддреживают маски (В stable_baselines(stable_contrib) есть только PPO, который выполняет данное условие)

In [34]:
def mask_fn(env: gym.Env) -> np.ndarray:
    return env.action_masks()

env = OnlineEnvLab(config=test_config,model=model,train_loader=train_loader)
env = ActionMasker(env, mask_fn) 
PPOmodel = MaskablePPO("MultiInputPolicy", env, gamma=0.4, verbose=1)
PPOmodel.learn(total_timesteps=5_000, log_interval=1)

evaluate_policy(PPOmodel, env, n_eval_episodes=20, reward_threshold=90, warn=False)

obs, _ = env.reset()
while True:
    action_masks = get_action_masks(env)
    action, _states = PPOmodel.predict(obs, action_masks=action_masks)
    obs, reward, terminated, truncated, info = env.step(action)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


KeyboardInterrupt: 

## Дискретный конвейер без масок

### Сам env

Функционал тот же, что и у OnlineEnvConveyorContinous, только мы дискретизирум отрезок [0,1]

TODO: 
- Code review. 
- Снести кусок кода с обучением модели в отдельную функцию. 
- Придумать, что делать с info, ибо он пустой (я знаю, что там сейчас возвращается маска, но это можно убрать и ничего не должно поменяться). 
- Поменять train_loader на более общий датасет, если будем ещё модели имплементировать.
- Обернуть в vector_env для параллельных вычисленый
- PEP8
- Поэксперементировать с observation_space

In [ ]:
class OnlineEnvConveyorContinous_Discrete(gym.Env):
    def __init__(self, config, model, train_loader, bins=1000):
        super().__init__()

        self.hyperparameter_space = config["hyperparameters"]
        self.model_type = config["model_type"]

        self.model = model
        self.train_loader = train_loader

        self.cur_step = 0
        self.hyp_setup = np.zeros(len(self.hyperparameter_space))
        self.action_dict = {}

        self.aranged = np.arange(0, 1, 1/bins) # это совершенно точно надо оборачивать в параметр
        self.action_space = gym.spaces.Discrete(len(self.aranged))
        
        self.observation_space = gym.spaces.Dict({
            "hyperparameters": gym.spaces.Box(low=-1, high=float("inf"), shape=(len(self.hyperparameter_space),), dtype=np.int32),
            "step": gym.spaces.Box(low=0, high=len(self.hyperparameter_space), shape=(1,), dtype=np.int32)
            })

    def _get_obs(self):
        return {"hyperparameters": self.hyp_setup, 
                "step": np.array([self.cur_step], dtype=np.int32)}
    
    def _get_info(self):
        return {}
    
    def step(self, action):
        terminated = self.cur_step >= len(self.hyp_setup)
        if not terminated:
            self.hyp_setup[self.cur_step] = self.interpret_action(action)
        if self.model_type == "cnn":
            if terminated:
                key_order = ["optimizer", "learning_rate", "batch_size", "number_epochs", "criterion"]
                optimizer, learning_rate, batch_size, number_epochs, criterion = (self.action_dict[key] for key in key_order)
                optimizer = optimizer(self.model.parameters(), lr=learning_rate)
                for epoch in range(number_epochs):  
                    running_loss = 0.0
                    for i, data in enumerate(self.train_loader, 0):
                        inputs, labels = data

                        optimizer.zero_grad()

                        outputs = self.model(inputs)
                        loss = criterion(outputs, labels)
                        loss.backward()
                        optimizer.step()

                        running_loss += loss.item()
                        if i % batch_size == batch_size-1:
                            running_loss = 0.0 
                print(f"CNN training finished. Final loss: {running_loss}")
                reward = -running_loss
            else:
                reward = 0
        self.cur_step+=1
        observation = self._get_obs()
        info = self._get_info()
        truncated = False
        return observation, reward, terminated, truncated, info
        
    def reset(self, seed: Optional[int] = None):
        super().reset(seed=seed)
        
        self.hyp_setup = np.zeros(len(self.hyperparameter_space), dtype=np.int32)
        self.cur_step = 0
        self.action_dict = {}

        observation = self._get_obs()
        info = self._get_info()
        return observation, info
    
    def interpret_action(self, action):
        if self.model_type == "cnn":
            name = self.hyperparameter_space[self.cur_step]["name"]
            if name in ["optimizer", "batch_size", "number_epochs", "criterion"]:
                num_options = len(self.hyperparameter_space[self.cur_step]["options"])
                filtered_action = min(math.floor(num_options * self.aranged[action]), num_options - 1)
                self.action_dict[name] = self.hyperparameter_space[self.cur_step]["options"][filtered_action]
                return filtered_action
            elif name == "learning_rate":
                
                lr_value = float(self.aranged[action])
                print(lr_value)
                self.action_dict[name] = lr_value
                return lr_value

### Модель для обучения

Работает с любыми алгоритмами

In [ ]:
env = OnlineEnvConveyorContinous_Discrete(config=test_config,model=model,train_loader=train_loader)
model_DQN = DQN("MultiInputPolicy", env, verbose=1)
model_DQN.learn(total_timesteps=10000, log_interval=4)


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
0.6000000000000001
CNN training finished. Final loss: 7.9302649050951
0.5
CNN training finished. Final loss: 4.2501230062916875
0.4
CNN training finished. Final loss: 102.9102087020874
0.0
CNN training finished. Final loss: 97.95719480514526
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 6        |
|    ep_rew_mean      | -53.3    |
|    exploration_rate | 0.977    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 0        |
|    time_elapsed     | 68       |
|    total_timesteps  | 24       |
----------------------------------
0.8
CNN training finished. Final loss: 96.79941248893738
0.6000000000000001
CNN training finished. Final loss: 23.445345163345337
0.5
CNN training finished. Final loss: 98.15254473686218
0.5
CNN training finished. Final loss: 23.046159982681274
----------------------------------

KeyboardInterrupt: 